In [1]:
# GPTv2 모델 파인튜닝 & 저장 : 약 3시간 걸림 (8 CPU & 0 GPU)

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, AutoTokenizer
from timeit import default_timer

# Load the text data
with open('한글성경(마침표있음)_정제후말뭉치_오십만단어.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Instantiate the GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# Tokenize the text data
tokenized_text = tokenizer(text, return_tensors='tf')
print(tokenized_text)

# Define the training parameters
model_path = "./output/kogpt2-마침표있음-finetuned-epoch-66"
learning_rate = 3e-5
batch_size = 16
epochs = 66

# Define the training function
@tf.function
def train_step(input_ids):
    # Truncate input sequence
    max_seq_length = 1024 # "1024" for GPT-2
    input_ids = input_ids[:, :max_seq_length]
    with tf.GradientTape() as tape:
        outputs = model(input_ids, training=True)
        logits = outputs.logits[:, :-1, :]
        labels = input_ids[:, 1:]
        loss_value = loss(labels, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

# Create a TensorSliceDataset from the tokenized text
dataset = tf.data.Dataset.from_tensor_slices(tokenized_text['input_ids'])
dataset = dataset.batch(batch_size)

# Define the loss function and optimizer
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Fine-tune the model
for epoch in range(epochs):
    start = default_timer()
    print(f'Epoch {epoch+1}/{epochs}')
    for step, batch in enumerate(dataset):
        loss_value = train_step(batch)
        if step % 50 == 0:
            print(f'Step {step} Loss {loss_value}')
    end = default_timer()
    print("Time duration(in seconds):", end - start)

# Save the fine-tuned model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

2023-03-24 09:35:30.525249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-24 09:35:30.525278: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gbike/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-03-24 09:35:38.656290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-24 09:35:38.656319: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03

{'input_ids': <tf.Tensor: shape=(1, 908876), dtype=int32, numpy=array([[14549,  9347,   387, ...,  9050,  7523,   389]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 908876), dtype=int32, numpy=array([[1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}
Epoch 1/66
Step 0 Loss 3.3351941108703613
Time duration(in seconds): 14.322258066007635
Epoch 2/66
Step 0 Loss 3.1992506980895996
Time duration(in seconds): 4.187989727011882
Epoch 3/66
Step 0 Loss 2.6766788959503174
Time duration(in seconds): 4.144087071996182
Epoch 4/66
Step 0 Loss 2.4230310916900635
Time duration(in seconds): 4.115100048016757
Epoch 5/66
Step 0 Loss 2.120518207550049
Time duration(in seconds): 3.969981091999216
Epoch 6/66
Step 0 Loss 1.9098947048187256
Time duration(in seconds): 4.00973854699987
Epoch 7/66
Step 0 Loss 1.7090203762054443
Time duration(in seconds): 3.9106705269950908
Epoch 8/66
Step 0 Loss 1.5147719383239746
Time duration(in seconds): 3.9365966329933144
Epoch 9/66
Step 0 Loss 1.3599551916122437
Time dura

('kogpt2-finetuned-epoch-66/tokenizer_config.json',
 'kogpt2-finetuned-epoch-66/special_tokens_map.json',
 'kogpt2-finetuned-epoch-66/vocab.json',
 'kogpt2-finetuned-epoch-66/merges.txt',
 'kogpt2-finetuned-epoch-66/added_tokens.json',
 'kogpt2-finetuned-epoch-66/tokenizer.json')

In [2]:
# 모델 로드 및 테스트
from transformers import TFGPT2LMHeadModel, AutoTokenizer
from timeit import default_timer

print("[모델 로딩 시간]")
start = default_timer()

loaded_model = TFGPT2LMHeadModel.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")
loaded_tokenizer = AutoTokenizer.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")

end = default_timer()
print("Time duration(in seconds):", end - start)

print("[문장 생성 시간]")
start = default_timer()

prompt = "예수님"
input_ids = loaded_tokenizer.encode(prompt, return_tensors="tf")
output_ids = loaded_model.generate(input_ids=input_ids,
                                   max_length=200+input_ids.shape[1],
                                   num_beams=1, # Higher value increases the computational cost
                                   no_repeat_ngram_size=1, # Higher means more repetitive words
                                   repetition_penalty=2.0, # Higher avoids repeating sentences
                                   temperature=0.1, # Higher means more diverse : 0 ~ 1
                                   top_p=0.9, # Lower means more diverse : 0 ~ 1
                                   early_stopping=True)
generated_text = loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Prompt: " + prompt)

split_generated_text = generated_text.split(prompt)
if len(split_generated_text) > 1:
    generated_text = split_generated_text[1]
# Trim the sentences after the last period(.)
text_to_remove = generated_text.split('.')[-1]
generated_text = generated_text.replace(text_to_remove,'')
print("Generated text: " + generated_text)

end = default_timer()
print("Time duration(in seconds):", end - start)

[모델 로딩 시간]


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./output/kogpt2-마침표있음-finetuned-epoch-66.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Time duration(in seconds): 1.886370230000466
[문장 생성 시간]
Prompt: 예수님
Generated text: 께서 또 말씀하셨습니다. 빛이 생겨라! 그러자 빛과 어둠이 생겼습니다.
그 때에 하나님의 영은 물 위에서 움직이고 계셔서 물을 둘로 나누어라.
하나님은 빛을 낮 이라 부르시고, 그 빛들이 밤 이브라힘을 만드셨다.
저녁과 아침, 그리고 저녁은 하늘과 땅 위에 놓여 있었습니다
(출애굽기 25:12). 새벽 두 시경, 안식일, 그리하여 하늘 아래의 모든 것들이 다 지어졌지요.
그러나 해가 뉘엿뉘그레지고 날이 되니, 이제야 일이 풀렸네요.
일곱째 날, 한빛님이 하시던 일을 마치시고 쉬었 았다 가겠어요.
한빛이 일곱 번째 날이었죠.
처음 창조된 남자, 하늘에 빛나는 별들을 보시기에 좋았는데요. 오늘도 큰 추위가 이어지겠지만, 내일은 다시 영하로 떨어집니다.
내륙에는 안개가 올라와 온 나라를 뒤흔들 것으로 보입니다.
자세한 날씨 기상캐스터 연결해서 알아보도록 하겠니다.
김민혜 캐s. 나와 있습니다.
네.
Time duration(in seconds): 64.62919808900915


In [3]:
# 모델 로드 및 테스트
from transformers import TFGPT2LMHeadModel, AutoTokenizer
from timeit import default_timer

print("[모델 로딩 시간]")
start = default_timer()

loaded_model = TFGPT2LMHeadModel.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")
loaded_tokenizer = AutoTokenizer.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")

end = default_timer()
print("Time duration(in seconds):", end - start)

print("[문장 생성 시간]")
start = default_timer()

prompt = "예수님"
input_ids = loaded_tokenizer.encode(prompt, return_tensors="tf")
output_ids = loaded_model.generate(input_ids=input_ids,
                                   max_length=200+input_ids.shape[1],
                                   early_stopping=True)
generated_text = loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Prompt: " + prompt)

split_generated_text = generated_text.split(prompt)
if len(split_generated_text) > 1:
    generated_text = split_generated_text[1]
# Trim the sentences after the last period(.)
text_to_remove = generated_text.split('.')[-1]
generated_text = generated_text.replace(text_to_remove,'')
print("Generated text: " + generated_text)

end = default_timer()
print("Time duration(in seconds):", end - start)

[모델 로딩 시간]


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./output/kogpt2-마침표있음-finetuned-epoch-66.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Time duration(in seconds): 2.1700137660081964
[문장 생성 시간]
Prompt: 예수님
Generated text: 께서 또 말씀하셨습니다. 빛이 생겨라! 그러자 빛이 생겼습니다.
그 빛이 하나님께서 보시기에 좋았습니다. 하나님께서 빛과 어둠을 나누셨습니다.
하나님께서는 빛을 낮 이라 부르시고, 어둠을 밤 이라 부르셨습니다. 저녁이 지나고 아침이 되니, 이 날이 첫째 날이었습니다.
하나님께서 또 말씀하셨습니다. 물 한가운데 둥근 공간이 생겨 물을 둘로 나누어라.
하나님께서 둥근 공간을 만드시고, 그 공간 아래의 물과 공간 위의 물을 나누시니 그대로 되었습니다.
하나님께서 그 공간을 하늘 이라 부르셨습니다. 저녁이 지나고 아침이 되니, 이 날이 둘째 날이었습니다.
하나님께서 말씀하셨습니다. 하늘 아래의 물은 한 곳으로 모이고 뭍은 드러나라 하시니 그대로 되었습니다.
하나님께서 뭍을 땅 이라 부르시고 모인 물은 바다 라고 부르셨습니다. 하나님께서 보시기에 좋았습니다.
하나님께서 말씀하셨습니다. 땅은 풀과 씨를 맺는 식물과 씨가 든 열매를 맺는 온갖 과일나무를 내어라 하시니, 그대로 되었습니다.
Time duration(in seconds): 50.014406750007765


In [4]:
# 모델 로드 및 테스트
from transformers import TFGPT2LMHeadModel, AutoTokenizer
from timeit import default_timer

print("[모델 로딩 시간]")
start = default_timer()

loaded_model = TFGPT2LMHeadModel.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")
loaded_tokenizer = AutoTokenizer.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")

end = default_timer()
print("Time duration(in seconds):", end - start)

print("[문장 생성 시간]")
start = default_timer()

prompt = "예수님"
input_ids = loaded_tokenizer.encode(prompt, return_tensors="tf")
output_ids = loaded_model.generate(input_ids=input_ids,
                                   max_length=30+input_ids.shape[1],
                                   early_stopping=True)
generated_text = loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Prompt: " + prompt)

split_generated_text = generated_text.split(prompt)
if len(split_generated_text) > 1:
    generated_text = split_generated_text[1]
# Trim the sentences after the last period(.)
text_to_remove = generated_text.split('.')[-1]
generated_text = generated_text.replace(text_to_remove,'')
print("Generated text: " + generated_text)

end = default_timer()
print("Time duration(in seconds):", end - start)

[모델 로딩 시간]


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./output/kogpt2-마침표있음-finetuned-epoch-66.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Time duration(in seconds): 1.8056226579938084
[문장 생성 시간]
Prompt: 예수님
Generated text: 께서 또 말씀하셨습니다. 빛이 생겨라! 그러자 빛이 생겼습니다.
그 빛이 하나님께서 보시기에 좋았습니다.
Time duration(in seconds): 7.202055873000063


In [ ]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

def ouputGPT(prompt=""):
    print("[문장 생성 시간]")
    start = default_timer()

    #prompt = "예수님"
    input_ids = loaded_tokenizer.encode(prompt, return_tensors="tf")
    output_ids = loaded_model.generate(input_ids=input_ids,
                                       max_length=30+input_ids.shape[1],
                                       early_stopping=True)
    generated_text = loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print("Prompt: " + prompt)

    split_generated_text = generated_text.split(prompt)
    if len(split_generated_text) > 1:
        generated_text = split_generated_text[1]
    # Trim the sentences after the last period(.)
    text_to_remove = generated_text.split('.')[-1]
    generated_text = generated_text.replace(text_to_remove,'')
    print("Generated text: " + generated_text)

    end = default_timer()
    print("Time duration(in seconds):", end - start)

# Let's chat for 10 lines
for step in range(10):
    prompt = input(">> User:")
    if prompt.lower() == "bye": break
    print("GPT: {}".format( ouputGPT(prompt) ))
    print_current_datetime()